In [19]:
# Imports
import pandas as pd
import plotly.express as px
from prettytable import PrettyTable
import plotly.graph_objects as go
import numpy as np
import random
import csv
import os
import json
from tqdm import tqdm
import altair as alt


In [53]:
# Local on Windows 10 box
WD = r"E:\BUSTEDS-MH"

#WD = os.path.join("D:", "BUSTEDS-MH")

print("# WD is:", WD)

# MAC OSX
#WD = "/Users/user/Documents/BUSTEDS-MH"

# Get directories
BUSTEDS_DIR = os.path.join(WD, "analysis", "Simulations_16_31", "31-seq", "BUSTEDS")
BUSTEDS_MH_DIR = os.path.join(WD, "analysis", "Simulations_16_31", "31-seq", "BUSTEDS-MH")

ER_Threshold = 5
pval_Threshold = 0.05

OUTPUT_CSV = os.path.join(WD, "tables", "Table_Simulation_31seq.csv")

# WD is: E:\BUSTEDS-MH


## Helper functions

In [54]:
def read_json(filename):
    #print("# Reading:", filename)
    if os.stat(filename).st_size == 0: 
        print("# File is empty:", filename)
        return []
    elif not os.path.exists(filename):
        print("# File does not exist")
        return []
    else:
        pass
    #end if
    with open(filename, "r") as fh:
        json_data = json.load(fh)
    fh.close()
    return json_data
#end method


#define function to calculate cv
#cv = lambda x: np.std(x, ddof=1) / np.mean(x) * 100 
#cv = lambda x: np.std(x) / np.mean(x)

In [4]:
BUSTEDS_MH_DIR_FILES = [os.path.join(BUSTEDS_MH_DIR, file.name) for file in os.scandir(BUSTEDS_MH_DIR) if file.name.endswith(".json")]
print("# Number of BUSTEDS-MH results:", len(BUSTEDS_MH_DIR_FILES))

BUSTEDS_DIR_FILES = [os.path.join(BUSTEDS_DIR, file.name) for file in os.scandir(BUSTEDS_DIR) if file.name.endswith(".json")]
print("# Number of BUSTEDS results:", len(BUSTEDS_DIR_FILES))


# Number of BUSTEDS-MH results: 14914
# Number of BUSTEDS results: 1157


In [5]:
# Simulations settings csv
SimSettings = os.path.join(WD, "data", "sim-key.csv")

df_settings = pd.read_csv(SimSettings)
df_settings

,sim_names,omega 1,omega 2,omega 3,omega 1 prop,omega 2 prop,omega 3 prop,N_mom2,N_mean,NRV,alpha 1,alpha 2,alpha 3,alpha 1 prop,alpha 2 prop,alpha 3 prop,S_mom2,S_mean,SRV,normalizer
0,lowerCV,0.01729,0.4031,2.077,0.8341,0.13612,0.02978,0.150836,0.131145,2.787491,1.000000,3.000000,8.000000,0.100000,0.700000,0.200000,19.200000,3.800000,0.574143,4.150000
1,lowerCVo1,0.01729,0.4031,1.000,0.8341,0.13612,0.02978,0.052147,0.099072,2.076762,1.000000,3.000000,8.000000,0.100000,0.700000,0.200000,19.200000,3.800000,0.574143,4.150000
2,lowerCVo6,0.01729,0.4031,6.000,0.8341,0.13612,0.02978,1.094447,0.247972,4.098636,1.000000,3.000000,8.000000,0.100000,0.700000,0.200000,19.200000,3.800000,0.574143,4.150000
3,noSynCV,0.01729,0.4031,2.077,0.8341,0.13612,0.02978,0.150836,0.131145,2.787491,1.000000,1.000000,1.000000,0.465754,0.445863,0.088383,1.000000,1.000000,0.000000,1.000000
4,noSynCVo1,0.01729,0.4031,1.000,0.8341,0.13612,0.02978,0.052147,0.099072,2.076762,1.000000,1.000000,1.000000,0.465754,0.445863,0.088383,1.000000,1.000000,0.000000,1.000000
5,noSynCVo6,0.01729,0.4031,6.000,0.8341,0.13612,0.02978,1.094447,0.247972,4.098636,1.000000,1.000000,1.000000,0.465754,0.445863,0.088383,1.000000,1.000000,0.000000,1.000000
6,CV1,0.01729,0.4031,2.077,0.8341,0.13612,0.02978,0.150836,0.131145,2.787491,0.580085,0.583271,3.515658,0.142817,0.654865,0.202318,2.771469,1.176092,1.001837,1.450346
7,CV1o1,0.01729,0.4031,1.000,0.8341,0.13612,0.02978,0.052147,0.099072,2.076762,0.580085,0.583271,3.515658,0.142817,0.654865,0.202318,2.771469,1.176092,1.001837,1.450346
8,CV1o6,0.01729,0.4031,6.000,0.8341,0.13612,0.02978,1.094447,0.247972,4.098636,0.580085,0.583271,3.515658,0.142817,0.654865,0.202318,2.771469,1.176092,1.001837,1.450346
9,CV5,0.01729,0.4031,2.077,0.8341,0.13612,0.02978,0.150836,0.131145,2.787491,0.033896,0.054996,7.770528,0.513549,0.467490,0.018961,1.146914,0.190457,5.533358,2.042793


## Process data

In [6]:
df_dict = {}

#BUSTEDS_MH_DIR_FILES_lowerCVo1 = [file for file in BUSTEDS_MH_DIR_FILES if "lowerCVo1" in str(file)]

#BUSTEDS_MH_DIR_FILES = [os.path.join(BUSTEDS_MH_DIR, file.name) for file in os.scandir(BUSTEDS_MH_DIR) if file.name.endswith(".json")]

#for item in tqdm(BUSTEDS_MH_DIR_FILES_lowerCVo1):

for item in tqdm(BUSTEDS_MH_DIR_FILES):
    basename = os.path.basename(item).replace(".BUSTEDS-MH.json", "")
    
    # Find BUSTEDS File
    #BUSTEDS_File = os.path.join(BUSTEDS_DIR, basename + ".nex.BUSTEDS.json")
    
    # Read 
    #print()
    json_data_BUSTEDS_MH = read_json(item)
    #json_data_BUSTEDS = read_json(BUSTEDS_File)
    if json_data_BUSTEDS_MH == []: continue
    
    #print("# Data loaded")
    df_dict[basename] = {"Sequences": json_data_BUSTEDS_MH["input"]["number of sequences"]}
    df_dict[basename].update({"Method": "BUSTEDS-MH"})
    df_dict[basename].update({"Codons": json_data_BUSTEDS_MH["input"]["number of sites"]})
    df_dict[basename].update({"LRT p-value": json_data_BUSTEDS_MH["test results"]["p-value"]})

    # cAIC
    df_dict[basename].update({"cAIC": json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["AIC-c"]})
    
    # CV of omega
    A = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["0"]["omega"] 
    B = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["1"]["omega"] 
    C = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Test"]["2"]["omega"] 
    #df_dict[basename].update({"CV(omega)": cv([A, B, C])})
    
    # CV of alpha
    D = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["0"]["rate"] 
    E = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["1"]["rate"] 
    F = json_data_BUSTEDS_MH["fits"]["Unconstrained model"]["Rate Distributions"]["Synonymous site-to-site rates"]["2"]["rate"] 
    #df_dict[basename].update({"CV(alpha)": cv([D, E, F])})
    
    # omega 3
    df_dict[basename].update({"omega_3": json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                              ["Rate Distributions"]["Test"]["2"]["omega"]})
    df_dict[basename].update({"proportion_3": json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                              ["Rate Distributions"]["Test"]["2"]["proportion"]})
    
    # DH rate, TH rate
    df_dict[basename].update({"DH_Rate": json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                              ["rate at which 2 nucleotides are changed instantly within a single codon"]})
    df_dict[basename].update({"TH_Rate": json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                              ["rate at which 3 nucleotides are changed instantly within a single codon"]})
    df_dict[basename].update({"TH_Rate_SI": json_data_BUSTEDS_MH["fits"]["Unconstrained model"]
                              ["rate at which 3 nucleotides are changed instantly within a single codon between synonymous codon islands"]})
    ER_SITES = []    
    ER_df_dict = {}
    
    if "constrained" in json_data_BUSTEDS_MH["Evidence Ratios"].keys():
        
        #print("# ER Constrained Sites:", len(json_data_BUSTEDS_MH["Evidence Ratios"]["constrained"][0]))
        
        for site, val in enumerate(json_data_BUSTEDS_MH["Evidence Ratios"]["constrained"][0]):
            if val > ER_Threshold:
                ER_SITES.append(str(site + 1))
                ER_df_dict[site + 1] = {"BUSTEDS-MH ER": val}
                
            #end if
        #end for
        
        # assert that there are more than 0 sites here.
        df_dict[basename].update({"num_ER_Sites":  int(len(ER_df_dict.keys()))})
        #print(ER_df_dict.keys())
    #end if 

# end for

  1%|▊                                                                             | 157/14914 [00:03<05:29, 44.78it/s]

# File is empty: D:\BUSTEDS-MH\analysis\Simulations_16_31\31-seq\BUSTEDS-MH\add_D_300_replicate.50.BUSTEDS-MH.json
# File is empty: D:\BUSTEDS-MH\analysis\Simulations_16_31\31-seq\BUSTEDS-MH\add_D_300_replicate.52.BUSTEDS-MH.json
# File is empty: D:\BUSTEDS-MH\analysis\Simulations_16_31\31-seq\BUSTEDS-MH\add_D_300_replicate.54.BUSTEDS-MH.json


  1%|▉                                                                             | 179/14914 [00:03<04:06, 59.78it/s]

# File is empty: D:\BUSTEDS-MH\analysis\Simulations_16_31\31-seq\BUSTEDS-MH\add_D_300_replicate.70.BUSTEDS-MH.json


  4%|██▊                                                                           | 530/14914 [00:22<08:42, 27.50it/s]

...

 94%|███████████████████████████████████████████████████████████████████████▍    | 14008/14914 [09:16<00:21, 42.95it/s]

# File is empty: D:\BUSTEDS-MH\analysis\Simulations_16_31\31-seq\BUSTEDS-MH\noSynCVo6_100_replicate.14.BUSTEDS-MH.json


 94%|███████████████████████████████████████████████████████████████████████▌    | 14042/14914 [09:16<00:12, 70.64it/s]

# File is empty: D:\BUSTEDS-MH\analysis\Simulations_16_31\31-seq\BUSTEDS-MH\noSynCVo6_100_replicate.60.BUSTEDS-MH.json


100%|████████████████████████████████████████████████████████████████████████████| 14914/14914 [09:45<00:00, 25.47it/s]


In [7]:
df_MH = pd.DataFrame.from_dict(df_dict, orient="index")
df_MH = df_MH.reset_index()
df_MH.index += 1
df_MH.rename(columns={'index': 'Filename'}, inplace = True)

#df_MH.to_csv("BUSTEDSMH_Simulations_31.csv")
df_MH

,Filename,Sequences,Method,Codons,LRT p-value,cAIC,omega_3,proportion_3,DH_Rate,TH_Rate,TH_Rate_SI,num_ER_Sites
1,add_D_100_replicate.1,31,BUSTEDS-MH,100,0.326708,6390.378508,1.336911,0.091431,0.000000,0.000000,0.000000,0.0
2,add_D_100_replicate.10,31,BUSTEDS-MH,100,0.442771,6353.634734,1.507306,0.048242,0.000000,0.000000,0.000000,0.0
3,add_D_100_replicate.100,31,BUSTEDS-MH,100,0.500000,6134.914092,1.001563,0.000000,0.000000,0.000000,0.000000,NaN
4,add_D_100_replicate.11,31,BUSTEDS-MH,100,0.065313,6316.573066,3.488098,0.025317,0.000000,0.000000,0.000000,4.0
5,add_D_100_replicate.12,31,BUSTEDS-MH,100,0.500000,6922.290291,1.001554,0.000000,0.289676,0.000000,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
14815,noSynCV_replicate.95,31,BUSTEDS-MH,1000,0.396044,64021.975582,1.125548,0.093851,0.000000,0.076875,0.000000,0.0
14816,noSynCV_replicate.96,31,BUSTEDS-MH,1000,0.199813,62900.278718,1.226251,0.080893,0.046990,0.000000,0.000000,0.0
14817,noSynCV_replicate.97,31,BUSTEDS-MH,1000,0.106552,61922.572894,2.017410,0.023660,0.000000,0.064478,0.000000,1.0
14818,noSynCV_replicate.98,31,BUSTEDS-MH,1000,0.049568,61899.395981,2.145090,0.022892,0.000000,0.000000,0.000000,2.0


In [8]:
# Assign CV(alpha) values to the df_MH file.


#for index, row in df_MH.iterrows():
#    print(row["Filename"])

df_MH["omega3_setting"] = ""
df_MH["SRV_setting"] = ""

for index, row in df_settings.iterrows():
    print(row["sim_names"], row["omega 3"], row["SRV"])
    search_item = row["sim_names"]
    # Loop over the df_MH files
    for index2, row2 in df_MH.iterrows():
        if search_item in row2["Filename"]:
            #then
            df_MH["omega3_setting"][index2] = row["omega 3"]
            df_MH["SRV_setting"][index2] = row["SRV"]
        #end if
    #end inner for
#end for


    
    

lowerCV 2.077 0.574142743


<ipython-input-8-08223a955c76>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_MH["omega3_setting"][index2] = row["omega 3"]
<ipython-input-8-08223a955c76>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_MH["SRV_setting"][index2] = row["SRV"]


lowerCVo1 1.0 0.574142743
lowerCVo6 6.0 0.574142743
noSynCV 2.077 0.0
noSynCVo1 1.0 0.0
noSynCVo6 6.0 0.0
CV1 2.077 1.001837059
CV1o1 1.0 1.001837059
CV1o6 6.0 1.001837059
CV5 2.077 5.533358493
CV5o1 1.0 5.533358493
CV5o6 6.0 5.533358493
CV3 2.077 3.018230747
CV3o1 1.0 3.018230747
CV3o6 6.0 3.018230747
allOis1_D 1.0 0.0
allOis1_B 1.0 0.574142743
allOis1_E 1.0 1.001837059
allOis1_N 1.0 3.018230747
allOis1_F 1.0 5.533358493
fill_C 2.077 0.302941165
fill_G 1.0 0.302941165
fill_K 6.0 0.302941165
CV0d1 2.077 0.102741156
CV0d1o1 1.0 0.102741156
CV0d1o6 6.0 0.102741156
add_D 2.077 0.34608458
add_E 1.0 0.34608458
add_F 6.0 0.34608458
add_G 2.077 0.5
add_H 1.0 0.5
add_I 6.0 0.5


In [9]:
df_MH

,Filename,Sequences,Method,Codons,LRT p-value,cAIC,omega_3,proportion_3,DH_Rate,TH_Rate,TH_Rate_SI,num_ER_Sites,omega3_setting,SRV_setting
1,add_D_100_replicate.1,31,BUSTEDS-MH,100,0.326708,6390.378508,1.336911,0.091431,0.000000,0.000000,0.000000,0.0,2.077,0.346085
2,add_D_100_replicate.10,31,BUSTEDS-MH,100,0.442771,6353.634734,1.507306,0.048242,0.000000,0.000000,0.000000,0.0,2.077,0.346085
3,add_D_100_replicate.100,31,BUSTEDS-MH,100,0.500000,6134.914092,1.001563,0.000000,0.000000,0.000000,0.000000,NaN,2.077,0.346085
4,add_D_100_replicate.11,31,BUSTEDS-MH,100,0.065313,6316.573066,3.488098,0.025317,0.000000,0.000000,0.000000,4.0,2.077,0.346085
5,add_D_100_replicate.12,31,BUSTEDS-MH,100,0.500000,6922.290291,1.001554,0.000000,0.289676,0.000000,0.000000,NaN,2.077,0.346085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14815,noSynCV_replicate.95,31,BUSTEDS-MH,1000,0.396044,64021.975582,1.125548,0.093851,0.000000,0.076875,0.000000,0.0,2.077,0.0
14816,noSynCV_replicate.96,31,BUSTEDS-MH,1000,0.199813,62900.278718,1.226251,0.080893,0.046990,0.000000,0.000000,0.0,2.077,0.0
14817,noSynCV_replicate.97,31,BUSTEDS-MH,1000,0.106552,61922.572894,2.017410,0.023660,0.000000,0.064478,0.000000,1.0,2.077,0.0
14818,noSynCV_replicate.98,31,BUSTEDS-MH,1000,0.049568,61899.395981,2.145090,0.022892,0.000000,0.000000,0.000000,2.0,2.077,0.0


## Save to file

In [51]:
#df_MH.to_csv("BUSTEDSMH_Simulations_31_withSettings.csv")
#OUTPUT_CSV
df_MH.to_csv(OUTPUT_CSV)

## Plots (Make this a View)

In [11]:
df_MH_omega3_1 = df_MH[df_MH["omega3_setting"] == 1.0]
df_MH_omega3_1

,Filename,Sequences,Method,Codons,LRT p-value,cAIC,omega_3,proportion_3,DH_Rate,TH_Rate,TH_Rate_SI,num_ER_Sites,omega3_setting,SRV_setting
497,add_E_100_replicate.1,31,BUSTEDS-MH,100,0.500000,6014.838947,1.002329,0.000000,0.0,0.000000,0.0,NaN,1.0,0.346085
498,add_E_100_replicate.10,31,BUSTEDS-MH,100,0.317312,5800.830878,13.187396,0.001140,0.0,0.000000,0.0,0.0,1.0,0.346085
499,add_E_100_replicate.12,31,BUSTEDS-MH,100,0.500000,5846.252482,1.000000,0.015492,0.0,0.432713,0.0,NaN,1.0,0.346085
500,add_E_100_replicate.15,31,BUSTEDS-MH,100,0.500000,5889.254653,1.000000,0.047276,0.0,0.000000,0.0,NaN,1.0,0.346085
501,add_E_100_replicate.17,31,BUSTEDS-MH,100,0.500000,5514.819920,1.000000,0.065295,0.0,0.000000,0.0,NaN,1.0,0.346085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13898,noSynCVo1_replicate.95,31,BUSTEDS-MH,1000,0.500000,60123.397182,1.002331,0.000000,0.0,0.071099,0.0,NaN,1.0,0.0
13899,noSynCVo1_replicate.96,31,BUSTEDS-MH,1000,0.500000,61445.193227,1.000000,0.029533,0.0,0.000000,0.0,NaN,1.0,0.0
13900,noSynCVo1_replicate.97,31,BUSTEDS-MH,1000,0.500000,60407.662728,1.000000,0.017111,0.0,0.000000,0.0,NaN,1.0,0.0
13901,noSynCVo1_replicate.98,31,BUSTEDS-MH,1000,0.500000,60368.005628,1.000000,0.043846,0.0,0.000000,0.0,NaN,1.0,0.0


In [12]:
df_MH_omega3_2_077 = df_MH[df_MH["omega3_setting"] == 2.077]
df_MH_omega3_2_077

,Filename,Sequences,Method,Codons,LRT p-value,cAIC,omega_3,proportion_3,DH_Rate,TH_Rate,TH_Rate_SI,num_ER_Sites,omega3_setting,SRV_setting
1,add_D_100_replicate.1,31,BUSTEDS-MH,100,0.326708,6390.378508,1.336911,0.091431,0.000000,0.000000,0.000000,0.0,2.077,0.346085
2,add_D_100_replicate.10,31,BUSTEDS-MH,100,0.442771,6353.634734,1.507306,0.048242,0.000000,0.000000,0.000000,0.0,2.077,0.346085
3,add_D_100_replicate.100,31,BUSTEDS-MH,100,0.500000,6134.914092,1.001563,0.000000,0.000000,0.000000,0.000000,NaN,2.077,0.346085
4,add_D_100_replicate.11,31,BUSTEDS-MH,100,0.065313,6316.573066,3.488098,0.025317,0.000000,0.000000,0.000000,4.0,2.077,0.346085
5,add_D_100_replicate.12,31,BUSTEDS-MH,100,0.500000,6922.290291,1.001554,0.000000,0.289676,0.000000,0.000000,NaN,2.077,0.346085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14815,noSynCV_replicate.95,31,BUSTEDS-MH,1000,0.396044,64021.975582,1.125548,0.093851,0.000000,0.076875,0.000000,0.0,2.077,0.0
14816,noSynCV_replicate.96,31,BUSTEDS-MH,1000,0.199813,62900.278718,1.226251,0.080893,0.046990,0.000000,0.000000,0.0,2.077,0.0
14817,noSynCV_replicate.97,31,BUSTEDS-MH,1000,0.106552,61922.572894,2.017410,0.023660,0.000000,0.064478,0.000000,1.0,2.077,0.0
14818,noSynCV_replicate.98,31,BUSTEDS-MH,1000,0.049568,61899.395981,2.145090,0.022892,0.000000,0.000000,0.000000,2.0,2.077,0.0


In [13]:
df_MH_omega3_6 = df_MH[df_MH["omega3_setting"] == 6]
df_MH_omega3_6

,Filename,Sequences,Method,Codons,LRT p-value,cAIC,omega_3,proportion_3,DH_Rate,TH_Rate,TH_Rate_SI,num_ER_Sites,omega3_setting,SRV_setting
953,add_F_100_replicate.10,31,BUSTEDS-MH,100,0.052662,6746.019696,7.214238,0.021388,0.215292,0.000000,0.000000,11.0,6.0,0.346085
954,add_F_100_replicate.100,31,BUSTEDS-MH,100,0.000128,6994.526077,4.602117,0.047824,0.000000,0.000000,0.000000,19.0,6.0,0.346085
955,add_F_100_replicate.11,31,BUSTEDS-MH,100,0.019270,6561.937660,3.783140,0.047972,0.000000,0.000000,0.000000,9.0,6.0,0.346085
956,add_F_100_replicate.15,31,BUSTEDS-MH,100,0.000239,6903.055648,7.492380,0.031244,0.000000,0.000000,0.000000,25.0,6.0,0.346085
957,add_F_100_replicate.17,31,BUSTEDS-MH,100,0.000065,7369.304428,4.000375,0.060434,0.037225,0.000000,0.000000,19.0,6.0,0.346085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14350,noSynCVo6_replicate.95,31,BUSTEDS-MH,1000,0.000000,68491.842670,5.733809,0.033093,0.000000,0.000000,0.000000,193.0,6.0,0.0
14351,noSynCVo6_replicate.96,31,BUSTEDS-MH,1000,0.000000,67971.856021,7.795953,0.022852,0.010254,0.000000,0.322584,122.0,6.0,0.0
14352,noSynCVo6_replicate.97,31,BUSTEDS-MH,1000,0.000000,67849.376073,3.571465,0.051991,0.000000,0.107058,0.000000,150.0,6.0,0.0
14353,noSynCVo6_replicate.98,31,BUSTEDS-MH,1000,0.000000,69543.068708,5.126726,0.034733,0.000000,0.000000,0.000000,159.0,6.0,0.0


## Process these new dataframes for plotting

In [23]:
def process_viz(df, label):
    global pval_Threshold
    y = set(df["SRV_setting"])
    #c = set(df["Codons"])
    df_dict = {}
    count = 0
    for value in y:
        df_holder = df[df["SRV_setting"] == value]
        for codons in set(df_holder["Codons"]):
            df_holder2 = df_holder[df_holder["Codons"] == codons]
            a =  df_holder2.shape[0]
            b =  df_holder2[df_holder2["LRT p-value"] <= pval_Threshold].shape[0]
            fraction = (b / a)
                    
            df_dict[count] = {
             "Codons" : str(codons),
             label : round(fraction, 3),
             "CV(a)": str(round(value,2))
            }
            count += 1
        #end for
    #end for
    # process to df
    df_df_dict = pd.DataFrame.from_dict(df_dict, orient="index")
    df_df_dict.index += 1
    return df_df_dict
#end method

In [24]:
df_2_077 = process_viz(df_MH_omega3_2_077, "Power")
#df_2_077


In [25]:
df_1 = process_viz(df_MH_omega3_1, "FPR")
#df_1

In [26]:
df_6 = process_viz(df_MH_omega3_6, "Power")
#df_2_077


## Visualization

In [46]:
source = df_1

#order = ["100", "300", "500", "1000", "5000"]
order = ["5000", "1000", "500", "300", "100"]

chart1 = alt.Chart(source).mark_line(
    point=alt.OverlayMarkDef(color="red")
).encode(
    x='CV(a)',
    y= alt.Y('FPR:Q',
        scale=alt.Scale(domain=(0, 1))
    ),
    color=alt.Color('Codons', sort=order)
).properties(
    width=300,
    height=150,
    title="BUSTEDS-MH on Simulated data, w3=1"
)

chart1

alt.Chart(...)

In [47]:
source = df_2_077

chart207 = alt.Chart(source).mark_line(
    point=alt.OverlayMarkDef(color="red")
).encode(
    x='CV(a)',
    y='Power',
    color=alt.Color('Codons', sort=order)
).properties(
    width=300,
    height=150,
    title="BUSTEDS-MH on Simulated data, w3=2.077"
)

chart207

alt.Chart(...)

In [48]:
source = df_6

chart6 = alt.Chart(source).mark_line(
    point=alt.OverlayMarkDef(color="red")
).encode(
    x='CV(a)',
    y='Power',
    color=alt.Color('Codons', sort=order)
).properties(
    width=300,
    height=150,
    title="BUSTEDS-MH on Simulated data, w3=6"
)

chart6

alt.Chart(...)

In [49]:
#alt.vconcat(chart1, chart207, chart6) | alt.vconcat(chart1, chart207, chart6)

alt.vconcat(chart1, chart207, chart6)

alt.VConcatChart(...)